In [49]:
import csv
import datetime
import glob
import json
import os
import requests
import urllib

if not os.path.isfile('doilog.csv'):
    with open('doilog.csv', 'w') as csvfile:
        logwriter = csv.writer(csvfile)
        logwriter.writerow([ 'timestamp', 'owner', 'instrument', 'filename', 'doi', 'action' ])

with open('datacite-api-config.json') as f:
    data = json.load(f)
    username = data["username"]
    password = data["password"]

use_test_api = False

if use_test_api:
    api_endpoint = 'https://api.test.datacite.org/dois'
else:
    api_endpoint = 'https://api.datacite.org/dois'

In [31]:
#Retrieve Metadata 

dois = ['10.34863/sgcx-yv30']

path = 'downloaded'
if not os.path.exists(path):
    os.mkdir(path)

url = api_endpoint
headers = {
    'Content-Type': 'application/vnd.api+json',
}
print('Bulk downloading metadata for DOIs in array')
for doi in dois:
    url = api_endpoint + '/' + doi
    print('Getting metadata for ' + doi)
    response = requests.request("GET", url, auth=(username, password), headers = headers)
    data = json.loads(response.text)
    prefix = doi.split('/')[0]
    suffix = doi.split('/')[1]
    print('Writing to ' + path + '/'+ prefix + '/' + suffix + '.json')
    if not os.path.exists(path + '/'+ prefix):
        os.mkdir(path + '/'+ prefix)
    with open(path + '/'+ prefix + '/' + suffix + '.json', 'w+') as jsonfile:
        json.dump(data, jsonfile, indent=3)
print('Done')

Bulk downloading metadata for DOIs in array
Getting metadata for 10.34863/sgcx-yv30
Writing to downloaded/10.34863/sgcx-yv30.json
Done


In [59]:
import pandas as pd
import requests
import json
import datetime
import csv
import os

def check_if_instrument_minted(instrument_name, log_file='doilog.csv'):
    if not os.path.isfile(log_file):
        return False 
    df = pd.read_csv(log_file)
    return not df[df['instrument'].str.contains(instrument_name, case=False, na=False)].empty

path = 'TEST.xlsx'
headers = {
    'Content-Type': 'application/vnd.api+json',
}

print('Bulk minting DOIs for files in ' + path)

# Read Excel file
try:
    df = pd.read_excel(path)
except FileNotFoundError:
    print(f"File {path} not found.")
    exit()

for index, row in df.iterrows():
    # Check if the instrument has already been minted
    if check_if_instrument_minted(row['Name of Instrument']):
        print(f"DOI already minted for {row['Name of Instrument']}. Skipping...")
        continue  # Skip this row and continue with the next one

    # Construct the data payload for each record if not already minted
    data_payload = {
        "data": {
            "type": "dois",
            "attributes": {
                "prefix": "10.34863",
                "identifiers": [
                    {
                        "identifier": "1234567",
                        "identifierType": "SerialNumber"
                    }
                ],
                "creators": [
                    {
                        "nameIdentifiers": [
                            {
                                "schemeUri": "https://orcid.org",
                                "nameIdentifier": "https://orcid.org/0000-0002-2292-180X",
                                "nameIdentifierScheme": "ORCID"
                            }
                        ]
                    }
                ],
                "titles": [
                    {
                        "title": row['Name of Instrument']
                    }
                ],
                "Provider": {
                    "data": {
                        "id": row['Owner'],
                    }
                },
                "dates": [
                    {
                        "date": row['Date'],
                        "dateType": "Installed"
                    }
                ],
                "descriptions": [
                    {
                        "description": row['Description'],
                        "descriptionType": "Abstract" 
                    }
                ],
                "types": {
                    "resourceType": row['Instrument Type'],
                    "resourceTypeGeneral": "Instrument"
                },
    
    
            }
        }
    }

    # Convert the payload to JSON
    data = json.dumps(data_payload)

    # Send the POST request
    response = requests.post(api_endpoint, auth=(username, password), headers=headers, data=data)
    if response.status_code == 201:
        doi = response.json()['data']['id']
        timestamp = datetime.datetime.now().isoformat()
        print(f"DOI {doi} minted for {row['Name of Instrument']}")
        # Log success to CSV
        with open('doilog.csv', 'a', newline='') as csvfile:
            logwriter = csv.writer(csvfile)
            logwriter.writerow([timestamp, row['Owner'], row['Name of Instrument'], row['Identifier'], doi, 'created'])
    else:
        print(f"Failed to mint DOI for {row['Name of Instrument']}: {response.text}")


Bulk minting DOIs for files in TEST.xlsx
DOI already minted for F200. Skipping...
DOI already minted for GrandARM. Skipping...
DOI 10.34863/rgn4-3j98 minted for FIBSEM
